In [1]:
import torch
import torch.nn as nn
import os
import librosa
import numpy as np
import pandas as pd

In [2]:
class CNN(nn.Module):
    def __init__(self,in_channels=1, num_classes=4):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(8192, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.bn1(self.conv1(x))))
        x = self.pool(nn.functional.relu(self.bn2(self.conv2(x))))
        x = self.pool(nn.functional.relu(self.bn3(self.conv3(x))))
        x = self.pool(nn.functional.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 4):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1), #7，2，3
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
#ensemble 
def predict_audio_test(test_dir, device, model, epoch):
    ids = []
    predictions = []

    for filename in os.listdir(test_dir):
        song_id = int(filename[:-4])
        song_path = os.path.join(test_dir, filename)
        y, sr = librosa.load(song_path)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spec_normalized = (mel_spec_db - (-50.217636)) / 16.921597
        mel_tensor = torch.tensor(mel_spec_normalized).unsqueeze(0)
        with torch.no_grad():
            input = mel_tensor.to(device)
            output = model(input.unsqueeze(0))
            _, predicted = torch.max(output, 1)

        ids.append(song_id)
        predictions.append(predicted.item())

    submission_df = pd.DataFrame({'id': ids, 'category': predictions})
    submission_df.to_csv(f'test_predictions_{epoch}.csv', index=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#define the model
model1 = CNN()
model2 = ResNet(ResidualBlock, [3,4,6,3], num_classes=4).to(device) 

# Load state dicts
model1.load_state_dict(torch.load('CNN.pth'))
model2.load_state_dict(torch.load('Resnet.pth'))
model1.to(device)
model2.to(device)
model1.eval()
model2.eval()

test_dir='test_aug'
ids=[]
predictions=[]
weights = torch.tensor([0.5, 0.5], device=device)

for filename in os.listdir(test_dir):
    song_id = int(filename[:-4])
    song_path = os.path.join(test_dir, filename)
    y, sr = librosa.load(song_path)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_normalized = (mel_spec_db - (-50.217636)) / 16.921597
    mel_tensor = torch.tensor(mel_spec_normalized).unsqueeze(0)
    with torch.no_grad():
        inputs = mel_tensor.to(device).unsqueeze(0)
        # Get predictions from each model
        outputs_cnn = torch.softmax(model1(inputs), dim=1) * weights[0]
        outputs_resnet = torch.softmax(model2(inputs), dim=1) * weights[1]
        outputs_ensemble = outputs_cnn + outputs_resnet 
        _, predicted = torch.max(outputs_ensemble.data, 1)

        ids.append(song_id)
        predictions.append(predicted.item())

submission_df = pd.DataFrame({'id': ids, 'category': predictions})
submission_df.to_csv(f'extract_csv/test_predictions_ensemble.csv', index=False)